In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import openpyxl as pyxl
import pandas as pd
import numpy as np

from excel_helpers import *
from db_entries import *

import re
from itertools import chain
# we only need the function datetime.datetime.now; we can now reference it as dt.now
from datetime import datetime as dt

import sqlite3 as sql

import logging
logger = logging.getLogger()

In [3]:
%%capture

db_filename = 'my-budget-dev-v1.sqlite'

%run nuclear_option.py $db_filename
%run database_setup.py $db_filename

db = sql.connect(db_filename)

In [4]:
excel_filename = 'GemeinsameBilanzierung_16_17__dev.xlsx'

current_sheet = 'August'
comment_sheet = 'august_'

# We take here the option data_only since for now we are not interested in the expression
wb = pyxl.load_workbook(excel_filename, data_only=True)


august = wb.get_sheet_by_name(current_sheet)
august_ = wb.copy_worksheet(august)
august_.title = comment_sheet
august_.sheet_state = 'hidden'

In [5]:
def autogenerate_database_comment_multiple_excel_ranges(filename, ranges,
                                                        comment_sheet) :
    try :
        ranges_string = ', '.join(ranges[:-1]) + \
                        (' and ' if len(ranges) > 1 else '') + ranges[-1]
    except IndexError :
        raise ValueError('You should provide at least one Excel range')
    return 'This entry was automatically generated from the excel file ' + \
           '{}. It is based on the cells {}. A note has been '.format(
               filename, ranges_string) + \
           'added to the respective cells in the sheet {}.'.format(comment_sheet.title)

def autogenerate_database_comment(filename, range_, comment_sheet) :
    return autogenerate_database_comment_multiple_excel_ranges(
                filename, [range_], comment_sheet)

def autogenerate_excel_comment(date, db_filename, the_id) :
    return 'On {} this cell was automatically read and '.format(date) + \
           'inserted into the database {}. The id ofthe entry is {}.'.format(
               date, db_filename, the_id)

In [6]:
def automatically_enter_payment_row(date, payment_type, description, amount, money_pot, budget_type,
                                    ranges, excel_filename, current_sheet, comment_sheet, db_filename) :

    list_of_ranges = [current_sheet.title + '!' + a_range for a_range in ranges]
    
    database_comment = autogenerate_database_comment_multiple_excel_ranges(
        excel_filename, list_of_ranges, comment_sheet.title)
    
    the_id = put_payment_into_database_autogenerated(db, date, payment_type, description, amount, 
                                                     money_pot, budget_type, database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    
    #Get a list of all referenced cells in the ranges
    cells = list(chain.from_iterable([list_from_range_string(a_range) for a_range in ranges]))
        
    put_comment_into_excel(comment_sheet, cells, excel_comment)
    
    return the_id

def automatically_enter_transfer_row(date, transfer_type, description, amount, money_pot_source, 
                                     money_pot_sink, ranges, excel_filename, current_sheet, 
                                     comment_sheet, db_filename, effect_date = None) :

    list_of_ranges = [current_sheet.title + '!' + a_range for a_range in ranges]
    
    database_comment = autogenerate_database_comment_multiple_excel_ranges(
        excel_filename, list_of_ranges, comment_sheet)
    
    the_id = put_transfer_into_database_autogenerated(db, date, transfer_type, description, amount, 
                                                      money_pot_source, money_pot_sink, comment=database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    
    #Get a list of all referenced cells in the ranges
    cells = list(chain.from_iterable([list_from_range_string(a_range) for a_range in ranges]))
        
    put_comment_into_excel(comment_sheet, cells, excel_comment)
    
    return the_id

In [7]:
def subset_data_frame(data_frame, rules) :
    """Subset a pandas.DataFrame by a dictionary of rules. The rules should have the following form:
    {<column name> : <restriction>} where <restriction> is either a number, a string starting with 
    '<' or '>' and followed by number or a general string. If it is a simple string or number, the 
    subsetting expression
        data_frame[dataframe[<column name>] == <restriction>]
    will be used. The other case will be interpreted as
        data_frame[dataframe[<column name>] <|> <number>].
    """
    
    result = data_frame
    for rule in rules.items() :
        column = rule[0]
        restriction = rule[1]
        
        if type(restriction) is str and restriction[0] in ['<', '>'] :
                try :
                    number = float(restriction[1:])
                except :
                    raise ValueError('After a rule starting with < or > you need to specify a number')
                result = result.loc[result[column] < number] if restriction[0] == '<' \
                            else result.loc[result[column] > number]
                continue

        result = result.loc[result[column] == restriction]
        if len(result) == 0 :
            logger.info('After the rule ' + str(rule) + ' no item was left')
            return result
    
    logger.info(str(len(result)) + ' items were selected.')
    return result

def multirule_subset(data_frame, rules) :
    """Collect multiple subsets (union of multiple subsets) of a pandas.DataFrame into a single DataFrame
    by using subset_data_frame.
    
    Keyword Arguments:
        data_frame -- the DataFrame to be subsetted
        rules      -- a list of dictionaries, where the dicts fulfill the requirements specified for 
                      subset_data_frame
    """
    new_data = pd.DataFrame(columns=data_frame.columns)
    for rule in rules :
        new_data = new_data.append(subset_data_frame(data_frame, rule))
        
    return new_data

In [8]:
budgeting = get_df_by_range(august_, 'A6', 'D130', 2)
budgeting.columns = ['budget_type', 'description', 'date', 'amount', 'excel_range']

col_titles = ['description', 'date', 'amount', 'excel_range']

max_bargeld = get_df_by_range(august_, 'H7', 'J130', 1)
max_bargeld.columns = col_titles
max_bargeld['money_pot'] = 'BM'

paul_bargeld = get_df_by_range(august_, 'K7', 'M130', 1)
paul_bargeld.columns = col_titles
paul_bargeld['money_pot'] = 'BP'

konto = get_df_by_range(august_, 'N7', 'P130', 1)
konto.columns = col_titles
konto['money_pot'] = 'KG'

conjoined = pd.concat([max_bargeld, paul_bargeld, konto])

In [9]:
all_info = pd.merge(budgeting, conjoined, how='outer', 
                    on=['description', 'date', 'amount'], indicator=True)

all_info['treated'] = 'No'

all_info['date'] = all_info['date'].dt.date
all_info['amount'] = all_info['amount'].round(2)

In [10]:
#with pd.option_context('display.max_rows', None):
#    display(all_info)

In [11]:
## Putting all the Info which had matching info on the left and the right hand of the budget sheet into
## the database.

## Only doing the "Barzahlung" payments right now

for BB in ['BM', 'BP'] :
    subsetting_rules = {'_merge' : 'both', 'money_pot' : BB,  'treated' : 'No'}

    full_match_bargeld = subset_data_frame(all_info, subsetting_rules)

    for index, row in full_match_bargeld.iterrows() :
        the_id = automatically_enter_payment_row(row['date'], 'Barzahlung', 
            row['description'], row['amount'], row['money_pot'], 
            row['budget_type'], [row['excel_range_x'], row['excel_range_y']],
            excel_filename, august, august_, db_filename)

        
        all_info.loc[index, 'treated'] = 'Yes' 

In [12]:
## For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
subsetting_rules = {'_merge' : 'both', 'money_pot' : 'KG',  'treated' : 'No'}
full_match_konto = subset_data_frame(all_info, subsetting_rules)
full_match_konto['payment_type'] = 'Kartenzahlung'
full_match_konto['use_data'] = 'X'
full_match_konto['complex'] = ''

In [13]:
gk_wb_filename = 'testtest.xlsx'

In [14]:
full_match_konto.to_excel(gk_wb_filename)
gk_wb = pyxl.load_workbook(gk_wb_filename, data_only=True)

gk_sheet = gk_wb.active

the_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')
        
the_list_a = ['Barzahlung', 'Kartenzahlung', 'Überweisung', 'Dauerauftrag',
            'SEPA-Mandat', 'Bankeinzug', 'Abheben', 'Kontotransfer', 
            'Bargeldtransfer', 'Einnahme', 'Geldfund']
the_list_b = ['B', 'K', 'U', 'D', 'S', 'BE', 'A', 'KT', 'BT', 'E', 'GF']

the_list_str = '"' + ','.join(the_list_a + the_list_b) + '"'

dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=the_list_str, allow_blank=False)
dv1.error ='Your entry is not in the list'
dv1.errorTitle = 'Invalid Entry'
dv1.prompt = 'Please select from the list'
dv1.promptTitle = 'List Selection'

dv2 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X"', allow_blank=True)
dv2.error ='Your entry is not in the list'
dv2.errorTitle = 'Invalid Entry'
dv2.prompt = 'Please check with an X or leave blank'
dv2.promptTitle = 'Checkmark'

dv1.ranges.append('K2:K' + str(gk_sheet.max_row))
dv2.ranges.append('L2:M' + str(gk_sheet.max_row))

gk_sheet.add_data_validation(dv1)
gk_sheet.add_data_validation(dv2)

for i in range(2, gk_sheet.max_row + 1) :
    gk_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    gk_sheet['K' + str(i)].fill = the_fill
    gk_sheet['L' + str(i)].fill = the_fill
    gk_sheet['M' + str(i)].fill = the_fill
    
for column_cells in gk_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    gk_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['F', 'G', 'H', 'I', 'J'] :
    gk_sheet.column_dimensions[row].hidden = True    
    
gk_wb.save(gk_wb_filename)

In [15]:
checked_full_match_konto = use_excel_for_data_entry(gk_wb_filename, copy_mode=False)

The file testtest.xlsx will be opened in Excel for data entry. Enter Q to abort, enter anything else to continue: 
Make any input to continue. Abort with Q: 
The read data is:


,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated,payment_type,use_data,complex
0,R,Miete,2017-08-01,-568.00,A6:D6,N13:P13,KG,both,No,Dauerauftrag,X,NaN
1,R,Miete FFM,2017-08-02,-450.00,A7:D7,N20:P20,KG,both,No,Überweisung,X,NaN
2,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.05,A8:D8,N12:P12,KG,both,No,SEPA-Mandat,X,NaN
3,R,Strom EnviaM,2017-08-30,-51.00,A9:D9,N89:P89,KG,both,No,SEPA-Mandat,X,NaN
7,R,Haftpflichtversicherung,2017-08-01,-7.50,A13:D13,N11:P11,KG,both,No,SEPA-Mandat,X,NaN
12,R,Handy Max,2017-08-10,-7.99,A18:D18,N41:P41,KG,both,No,SEPA-Mandat,NaN,X
14,G,Spotify Max,2017-08-15,-4.99,A20:D20,N52:P52,KG,both,No,SEPA-Mandat,X,NaN
15,G,Apple Music Paul,2017-08-25,-4.99,A21:D21,N76:P76,KG,both,No,Kartenzahlung,X,NaN
19,R,Monatskarte FFM,2017-07-31,-87.40,A25:D25,N9:P9,KG,both,No,Kartenzahlung,X,NaN
21,G,Fitnessstudio,2017-08-03,-39.80,A27:D27,N23:P23,KG,both,No,SEPA-Mandat,X,NaN



If you are not satisfied with the read data, enter any number. If you do this, the workbook will be opened again. Continue with any other input; abort with Q: 
OK.


In [16]:
miete_rules              = {'description' : 'Miete', 'amount' : -568,  'treated' : 'No'} 
haftpflicht_rules        = {'description' : 'Haftpflichtversicherung', 'amount' : -7.5,  'treated' : 'No'} 
berufsunfaehigkeit_rules = {'description' : 'Berufsunfähigkeitsversicherung', 'amount' : -49.05,  'treated' : 'No'} 
strom_rules_rules        = {'description' : 'Strom EnviaM', 'amount' : -51,  'treated' : 'No'} 
vodafone_rules           = {'description' : 'Vodafone', 'amount' : -19.99,  'treated' : 'No'} 
handy_paul_rules         = {'description' : 'Handy Paul', 'amount' : -7.99,  'treated' : 'No'} 
handy_max_rules          = {'description' : 'Handy Max', 'amount' : -7.99,  'treated' : 'No'} 
music_paul_rules         = {'description' : 'Apple Music Paul', 'amount' : -4.99,  'treated' : 'No'} 
music_max_rules          = {'description' : 'Spotify Max', 'amount' : -4.99,  'treated' : 'No'} 
backblaze_paul_rules     = {'description' : 'Backblaze Paul', 'amount' : -3.77,  'treated' : 'No'} 
backblaze_max_rules      = {'description' : 'Backblaze Max', 'amount' : -3.77,  'treated' : 'No'} 
semester_paul_rules      = {'description' : 'Semestergebühr Paul', 'amount' : -46.15,  'treated' : 'No'} 
semester_max_rules       = {'description' : 'Semestergebühr Max', 'amount' : -50.57,  'treated' : 'No'} 
sport_jahre_rules        = {'description' : '', 'amount' : -8,  'treated' : 'No'} 
sport_rules              = {'description' : 'Fitnessstudio', 'amount' : -39.8,  'treated' : 'No'} 
gew_rules                = {'description' : 'GEW', 'amount' : -2.5,  'treated' : 'No'} 
gez_rules                = {'description' : 'GEZ', 'amount' : -17.5,  'treated' : 'No'} 

all_rules = { 1 : [miete_rules                              ],
              2 : [haftpflicht_rules                        ],
              3 : [berufsunfaehigkeit_rules                 ],
              4 : [strom_rules_rules                        ],
              5 : [vodafone_rules                           ],
              6 : [handy_paul_rules                         ],
              7 : [handy_max_rules                          ],
              8 : [music_paul_rules                         ],
              9 : [music_max_rules                          ],
             10 : [backblaze_paul_rules, backblaze_max_rules],
             11 : [semester_paul_rules                      ],
             12 : [semester_max_rules                       ],
             13 : [sport_jahre_rules                        ],
             14 : [sport_rules                              ],
             15 : [gew_rules                                ],
             16 : [gez_rules                                ]}

monthly = [1, 2, 3, 4, 8, 9, 14]
bigger_intervalls = [10, 11, 12, 13, 15, 16]
variable_prices = [5, 6, 7]
## ToDo : 2 + 3 sonderfall beachten; insbesondere wird das gemeinsam vom konto abgebucht
## ToDo : 4 is only 51 € since mid 2017

for rule_number in monthly :
    rule = all_rules[rule_number][0]
    
    the_info = subset_data_frame(checked_full_match_konto, rule)

    for index, row in the_info.iterrows() :
        the_id = automatically_enter_payment_row(row['date'], row['payment_type'], 
            row['description'], row['amount'], row['money_pot'], 
            row['budget_type'], [row['excel_range_x'], row['excel_range_y']],
            excel_filename, august, august_, db_filename)

        crsr = db.cursor()
        crsr.execute('''INSERT INTO event_in_group VALUES (1, {});'''.format(the_id))
        db.commit()

        all_info.loc[index, 'treated'] = 'Yes'
        
## ToDo : do the bigger intervalls

In [17]:
for index, row in checked_full_match_konto.iterrows() :
    if checked_full_match_konto.loc[index, 'use_data'] != 'X' :
        continue
        
    the_id = automatically_enter_payment_row(row['date'], row['payment_type'], 
            row['description'], row['amount'], row['money_pot'], 
            row['budget_type'], [row['excel_range_x'], row['excel_range_y']],
            excel_filename, august, august_, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [18]:
## List of all fully matched things we didnt treat yet
subsetting_rules = {'_merge' : 'both', 'treated' : 'No'}
remaining_matched = subset_data_frame(all_info, subsetting_rules)

display(remaining_matched)

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
12,R,Handy Max,2017-08-10,-7.99,A18:D18,N41:P41,KG,both,No


In [19]:
display(all_info[all_info['description'].str.contains('Schatulle')])

subsetting_rules = {'treated' : 'xxx'}
schatulle_info = subset_data_frame(all_info, subsetting_rules)

for direction in ['an', 'aus'] :
    descriptor = 'Geld ' + direction + ' Schatulle'
    subsetting_rules = {'description' : descriptor, 'treated' : 'No'}
    schatulle_info = schatulle_info.append(subset_data_frame(all_info, subsetting_rules))
    
display(schatulle_info)

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
122,NaN,Geld aus Schatulle,2017-08-05,1.30,NaN,H10:J10,BM,right_only,No
126,NaN,Geld aus Schatulle,2017-08-20,0.50,NaN,H19:J19,BM,right_only,No
127,NaN,Schatullenkorrektur,2017-08-20,1.05,NaN,H20:J20,BM,right_only,No
128,NaN,Schatullenkorrektur,2017-08-20,-1.05,NaN,H21:J21,BM,right_only,No
131,NaN,Geld aus Schatulle,NaN,1.00,NaN,K7:M7,BP,right_only,No
132,NaN,Geld aus Schatulle,2017-08-08,1.35,NaN,K10:M10,BP,right_only,No


,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
122,NaN,Geld aus Schatulle,2017-08-05,1.30,NaN,H10:J10,BM,right_only,No
126,NaN,Geld aus Schatulle,2017-08-20,0.50,NaN,H19:J19,BM,right_only,No
131,NaN,Geld aus Schatulle,NaN,1.00,NaN,K7:M7,BP,right_only,No
132,NaN,Geld aus Schatulle,2017-08-08,1.35,NaN,K10:M10,BP,right_only,No


In [20]:
display(schatulle_info)
for index, row in schatulle_info.iterrows() :
    i = 0
    while True :
        the_date = all_info.loc[index+i, 'date']
        if str(the_date) == 'NaT' or the_date is np.nan :
            the_date = all_info.loc[index-i, 'date']
            if str(the_date) == 'NaT' or the_date is np.nan :
                i = i + 1
                continue
        
        break
        
    automatically_enter_transfer_row(
        the_date, 'Bargeldtransfer', row['description'], row['amount'], row['money_pot'], 'SB',
        [row['excel_range_y']], excel_filename, august, august_, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
122,NaN,Geld aus Schatulle,2017-08-05,1.30,NaN,H10:J10,BM,right_only,No
126,NaN,Geld aus Schatulle,2017-08-20,0.50,NaN,H19:J19,BM,right_only,No
131,NaN,Geld aus Schatulle,NaN,1.00,NaN,K7:M7,BP,right_only,No
132,NaN,Geld aus Schatulle,2017-08-08,1.35,NaN,K10:M10,BP,right_only,No


In [21]:
## The Three individual usages of the White Trianon Card : Charging
subsetting_rules = {'description' : 'Geld aufladen Trianon', 'amount' : -20, 'treated' : 'No'}
trianon_aufladen = subset_data_frame(all_info, subsetting_rules)

for index, row in trianon_aufladen.iterrows() :
    automatically_enter_transfer_row(
        row['date'], 'Aufladen', row['description'], row['amount'], row['money_pot'], 'CT',
        [row['excel_range_y']], excel_filename, august, august_, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [22]:
doppelter_kaffee = {'budget_type' : 'AM', 'description' : 'Doppelter Kaffee Trianon', 
                                     'amount' : -1.5, 'treated' : 'No'}
einfacher_kaffee = {'budget_type' : 'AM', 'description' : 'Kaffee', 
                                     'amount' : -0.75, 'treated' : 'No'}
rules = [doppelter_kaffee, einfacher_kaffee]
trianon_payments = multirule_subset(all_info, rules)

for index, row in trianon_payments.iterrows() :
    automatically_enter_payment_row(
        row['date'], 'Kartenzahlung', row['description'], row['amount'], 'CT', 
        row['budget_type'], [row['excel_range_x']], excel_filename, august, august_, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [23]:
subsetting_rules = {'budget_type' : 'AM', 'treated' : 'No'}
bundesbank_karte = subset_data_frame(all_info, subsetting_rules)

for index, row in bundesbank_karte.iterrows() :
    automatically_enter_payment_row(
        row['date'], 'Kartenzahlung', row['description'], row['amount'], 'CB', 
        row['budget_type'], [row['excel_range_x']], excel_filename, august, august_, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [24]:
subsetting_rules = {'description' : 'Geld abheben', '_merge' : 'right_only', 'treated' : 'No'}
geld_abheben = subset_data_frame(all_info, subsetting_rules)

subsetting_rules = {'description' : 'Geld abheben', 'amount' : '>0', '_merge' : 'right_only', 'treated' : 'No'}
geld_abheben_gotten = subset_data_frame(all_info, subsetting_rules)

subsetting_rules = {'description' : 'Geld abheben', 'amount' : '<0', '_merge' : 'right_only', 'treated' : 'No'}
geld_abheben_drawn = subset_data_frame(all_info, subsetting_rules)
geld_abheben_drawn['amount_abs'] = - geld_abheben_drawn['amount']

## Subselect relevant columns
abheben_gotten = geld_abheben_gotten[['date', 'amount', 'money_pot', 'description', 
                                      'excel_range_y']].reset_index(level=0)
abheben_drawn = geld_abheben_drawn[['date', 'amount_abs', 'money_pot', 
                                    'excel_range_y']].reset_index(level=0)


abheben_data = pd.merge(abheben_gotten, abheben_drawn, how='outer', left_on = ['date', 'amount'], 
                        right_on=['date', 'amount_abs'], left_index=True, indicator=True)

display(abheben_data)


for index, row in abheben_data[abheben_data['_merge'] == 'both'].iterrows() :
    automatically_enter_transfer_row(
        row['date'], 'Abheben', row['description'], row['amount'], row['money_pot_y'], row['money_pot_x'],
        [row['excel_range_y_x'], row['excel_range_y_y']], excel_filename, august, august_, db_filename)
    
    all_info.loc[row['index_x'], 'treated'] = 'Yes'
    all_info.loc[row['index_y'], 'treated'] = 'Yes'

,index_x,date,amount,money_pot_x,description,excel_range_y_x,index_y,amount_abs,money_pot_y,excel_range_y_y,_merge
1,120,2017-08-03,30.0,BM,Geld abheben,H8:J8,NaN,NaN,NaN,NaN,left_only
1,129,2017-08-22,40.0,BM,Geld abheben,H22:J22,146.0,40.0,KG,N68:P68,both
0,133,2017-08-21,20.0,BP,Geld abheben,K12:M12,145.0,20.0,KG,N65:P65,both


In [25]:
## For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
subsetting_rules = {'_merge' : 'right_only', 'treated' : 'No'}
remaining = subset_data_frame(all_info, subsetting_rules)
remaining['is_transfer'] = ''
remaining['transfer_type'] = ''
remaining['money_pot_source'] = remaining['amount'].apply(lambda x : 1 if x < 0 else 0) * \
                                remaining['money_pot'] 
remaining['money_pot_sink'] = remaining['amount'].apply(lambda x : 1 if x > 0 else 0) * \
                              remaining['money_pot']

In [26]:
mark_transfers_filename = 'testtest2.xlsx'
remaining.to_excel(mark_transfers_filename)
mark_transfers = pyxl.load_workbook(mark_transfers_filename, data_only=True)

remaining_sheet = mark_transfers.active

the_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')

dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X"', allow_blank=True)
dv1.error ='Your entry is not in the list'
dv1.errorTitle = 'Invalid Entry'
dv1.prompt = 'Please check with an X or leave blank'
dv1.promptTitle = 'Checkmark'
 
list_of_transfer_types = ['Abheben', 'Aufladen', 'Kontotransfer', 'Bargeldtransfer']
rule = '"' + ','.join(list_of_transfer_types) + '"'
dv2 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=rule, allow_blank=True)
dv2.error ='Your entry is not in the list'
dv2.errorTitle = 'Invalid Entry'
dv2.prompt = 'Please select from the list'
dv2.promptTitle = 'List Selection'

list_of_accounts = ['KG', 'KE', 'KM', 'KP', 'KB', 'KC', 'BM', 'BP', 'CB', 'CT', 'CM', 'GM', 'SB', 'SF']
rule = '"' + ','.join(list_of_accounts) + '"'
dv3 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=rule, allow_blank=True)
dv3.error ='Your entry is not in the list'
dv3.errorTitle = 'Invalid Entry'
dv3.prompt = 'Please select from the list'
dv3.promptTitle = 'List Selection'

dv1.ranges.append('K2:K' + str(remaining_sheet.max_row))
dv2.ranges.append('L2:L' + str(remaining_sheet.max_row))
dv3.ranges.append('M2:N' + str(remaining_sheet.max_row))

remaining_sheet.add_data_validation(dv1)
remaining_sheet.add_data_validation(dv2)
remaining_sheet.add_data_validation(dv3)

for i in range(2, remaining_sheet.max_row + 1) :
    remaining_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    remaining_sheet['K' + str(i)].fill = the_fill
    remaining_sheet['L' + str(i)].fill = the_fill
    remaining_sheet['M' + str(i)].fill = the_fill
    remaining_sheet['N' + str(i)].fill = the_fill
    
for column_cells in remaining_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    remaining_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['B', 'F', 'G', 'H', 'I', 'J'] :
    remaining_sheet.column_dimensions[row].hidden = True    
    
mark_transfers.save(mark_transfers_filename)

In [27]:
checked_data = use_excel_for_data_entry(mark_transfers_filename, copy_mode=False, open_before_read=True)
checked_transfer_data = checked_data[checked_data['is_transfer'] == 'X']

The file testtest2.xlsx will be opened in Excel for data entry. Enter Q to abort, enter anything else to continue: 
Make any input to continue. Abort with Q: 
The read data is:


,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated,is_transfer,transfer_type,money_pot_source,money_pot_sink
119,NaN,Geld aufladen Mitarbeiter ausweis,2017-08-01,-20.00,NaN,H7:J7,BM,right_only,No,NaN,NaN,BM,NaN
120,NaN,Geld abheben,2017-08-03,30.00,NaN,H8:J8,BM,right_only,No,NaN,NaN,NaN,BM
123,NaN,Betriebsausflug,2017-08-07,-18.50,NaN,H11:J11,BM,right_only,No,NaN,NaN,BM,NaN
124,NaN,Geldfund,2017-08-12,0.05,NaN,H12:J12,BM,right_only,No,NaN,NaN,NaN,BM
125,NaN,BBk Karte Aufladen,2017-08-14,-10.00,NaN,H15:J15,BM,right_only,No,NaN,NaN,BM,NaN
127,NaN,Schatullenkorrektur,2017-08-20,1.05,NaN,H20:J20,BM,right_only,No,NaN,NaN,NaN,BM
128,NaN,Schatullenkorrektur,2017-08-20,-1.05,NaN,H21:J21,BM,right_only,No,NaN,NaN,BM,NaN
130,NaN,Karte aufladen,2017-08-28,-10.00,NaN,H25:J25,BM,right_only,No,NaN,NaN,BM,NaN
134,NaN,Budgetbeitrag von BMS,2017-07-28,1468.00,NaN,N7:P7,KG,right_only,No,NaN,NaN,NaN,KG
135,NaN,Miete FFM auf Max Konto,2017-08-02,450.00,NaN,N19:P19,KG,right_only,No,NaN,NaN,NaN,KG



If you are not satisfied with the read data, enter any number. If you do this, the workbook will be opened again. Continue with any other input; abort with Q: 
OK.


/usr/local/lib/python3.6/site-packages/pandas/core/ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [ ]:
display(checked_transfer_data)

for index, row in checked_transfer_data.iterrows() :
    automatically_enter_transfer_row(
        row['date'], row['transfer_type'], row['description'], row['amount'], 
        row['money_pot_source'], row['money_pot_sink'],
        [row['excel_range_y']], excel_filename, august, august_, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
## Mark remaining as : Budget_Event, Money_Transfer, Payment, Complex Process
## Give the Opportunity to give provisorial ids

subsetting_rules = {'treated' : 'No'}
## If I do not use copy(), it will throw warnings when only 
## subsetting with one rule. I suppose it does copy automatically 
## after from the second subsetting
remaining = subset_data_frame(all_info, subsetting_rules).copy()
remaining['ignore'] = ''
remaining['is_budget_event'] = ''
remaining['is_payment'] = ''
remaining['is_transfer'] = ''
remaining['is_complex'] = ''
remaining['provisorial_id'] = ''

display(remaining)

In [ ]:
mark_remaining_filename = 'testtest3.xlsx'
remaining.to_excel(mark_remaining_filename)
mark_remaining = pyxl.load_workbook(mark_remaining_filename, data_only=True)

remaining_sheet = mark_remaining.active

yellow_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')

dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X"', allow_blank=True)
dv1.error ='Your entry is not in the list'
dv1.errorTitle = 'Invalid Entry'
dv1.prompt = 'Please check with an X or leave blank'
dv1.promptTitle = 'Checkmark'

dv1.ranges.append('K2:O' + str(remaining_sheet.max_row))
remaining_sheet.add_data_validation(dv1)

for i in range(2, remaining_sheet.max_row + 1) :
    remaining_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    remaining_sheet['K' + str(i)].fill = yellow_fill
    remaining_sheet['L' + str(i)].fill = yellow_fill
    remaining_sheet['M' + str(i)].fill = yellow_fill
    remaining_sheet['N' + str(i)].fill = yellow_fill
    remaining_sheet['O' + str(i)].fill = yellow_fill
    remaining_sheet['P' + str(i)].fill = yellow_fill
    
for column_cells in remaining_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    remaining_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['F', 'G', 'I', 'J'] :
    remaining_sheet.column_dimensions[row].hidden = True    
    
mark_remaining.save(mark_remaining_filename)

In [ ]:
checked_data = use_excel_for_data_entry(mark_remaining_filename, copy_mode=False, open_before_read=True)
checked_transfer_data = checked_data[checked_data['ignore'] != 'X']

In [ ]:
remaining = all_info[all_info['treated'] == 'No']

display(remaining)

print("number remaining: " + str(len(remaining)))

In [ ]:
#for table in ["money_events", "payments", "budget_events"] :
#    print('The table {}: '.format(table))
#    display(pd.read_sql_query('SELECT * FROM {};'.format(table), db))
with pd.option_context('display.max_rows', None):
    display(pd.read_sql_query(
        'SELECT * FROM payments LEFT JOIN money_events where money_events.id = payments.id', db))
    display(pd.read_sql_query(
        'SELECT * FROM transfers LEFT JOIN money_events where money_events.id = transfers.id', db))

In [ ]:
wb.save('GemBil.xlsx')

In [ ]:
db.close()